In [4]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [5]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Converting Cards data to Parquet Format").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.AssertionError: assertion failed: Expected hostname (not IP) but got 2402:e280:3e9d:c24:e07f:b6a0:2fb0:895e
	at scala.Predef$.assert(Predef.scala:170)
	at org.apache.spark.util.Utils$.checkHost(Utils.scala:1014)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:31)
	at org.apache.spark.executor.Executor.<init>(Executor.scala:184)
	at org.apache.spark.scheduler.local.LocalEndpoint.<init>(LocalSchedulerBackend.scala:59)
	at org.apache.spark.scheduler.local.LocalSchedulerBackend.start(LocalSchedulerBackend.scala:127)
	at org.apache.spark.scheduler.TaskSchedulerImpl.start(TaskSchedulerImpl.scala:183)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:501)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInst

In [ ]:
from pyspark.sql.types import *

#used "_" instead of space to match parqate file format
raw_schema = StructType([
    StructField("User", IntegerType(), True),
    StructField("CARD_INDEX", IntegerType(), True),
    StructField("Card_Brand", StringType(), True),
    StructField("Card_Type", StringType(), True),
    StructField("Card_Number", StringType(), True),
    StructField("Expires", StringType(), True),           
    StructField("CVV", StringType(), True),
    StructField("Has_Chip", StringType(), True),
    StructField("Cards_Issued", IntegerType(), True),
    StructField("Credit_Limit", StringType(), True),      
    StructField("Acct_Open_Date", StringType(), True),
    StructField("Year_PIN_Last_Changed", IntegerType(), True),
    StructField("Card_on_Dark_Web", StringType(), True)
])


In [ ]:
path = "file:///home/talentum/shared/dataSource/sd254_cards.csv"
df_raw = spark.read.csv(path, header = True, schema=raw_schema)
df_raw.show()

In [ ]:
df_raw.write.mode("overwrite").orc('/user/talentum/projectMaster/dataStaging/sd254_cards.parquet')